In [1]:
import numpy as np
import pandas as pd

import re
import datetime as dt

from sklearn.preprocessing import OneHotEncoder

# 0. Reading files
- dataset
- metadata of the dataset

In [2]:
df = pd.read_parquet('data/catB_train.parquet')
display(df)

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,...,recency_giclaim,giclaim_cnt_success,recency_giclaim_success,giclaim_cnt_unsuccess,recency_giclaim_unsuccess,flg_gi_claim_29d435_ever,flg_gi_claim_058815_ever,flg_gi_claim_42e115_ever,flg_gi_claim_856320_ever,f_purchase_lh
19550,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,1974-05-09,Female,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
4600,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,1979-11-11,Male,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
13337,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,1976-01-28,Male,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
15074,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,1976-03-19,Female,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
19724,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,1995-07-31,Female,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,3363260248,None,Singapore,P,LAPSED,2016-08-01,1994-05-24,Male,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
11964,c3509f0aef,Chinese,Singapore,P,ACTIVE,1990-03-20,1948-12-16,Male,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
5390,6e44cfb3e7,Chinese,Singapore,P,ACTIVE,2007-12-31,1967-12-06,Female,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
860,dd4e362f9f,Others,Singapore,P,ACTIVE,2007-09-01,1986-04-01,Male,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN


In [3]:
metadf = pd.read_csv("data/metadata.csv", encoding='cp1252')
display(metadf)

,column_name,is_numeric,is_date,to_delete,requires_preprocessing,remarks
0,affcon_visit_days,1,0,0,0.0,Metric related to client’s activity in affinit...
1,annual_income_est,0,0,0,1.0,This column is the client’s income bracket. Wi...
2,ape_507c37,1,0,0,0.0,NaN
3,ape_839f8a,1,0,0,0.0,NaN
4,ape_c4bda5,1,0,0,0.0,"This column is 0 for all, except 5 rows. But t..."
...,...,...,...,...,...,...
241,sumins_lh_f852af,1,0,0,NaN,NaN
242,sumins_ltc,1,0,0,NaN,NaN
243,sumins_ltc_43b9d5,1,0,0,NaN,NaN
244,tot_cancel_pols,1,0,0,NaN,NaN


# 1. Removing Unnecessary Columns
- Columns with constant values
- Columns with all NaN
- Columns that are aggregations of other columns:  
    1. ape_ltc = ape_grp_de05ae + ape_grp_6a5788 + ape_ltc_43b9d5

In [4]:
aggregated_cols = ["ape_ltc"]

In [5]:
n_unique_values_per_col = df.nunique(dropna=False).\
        reset_index().\
        rename(columns={"index" : "column", 0 : "n_unique_values"}).\
        sort_values("column").\
        sort_values("n_unique_values")

useless_cols_df = n_unique_values_per_col.\
        loc[n_unique_values_per_col.n_unique_values == 1].\
        sort_values("column").\
        reset_index(drop=True)

display(useless_cols_df)

,column,n_unique_values
0,ape_32c74c,1
1,ape_d0adeb,1
2,ape_gi,1
3,ape_gi_058815,1
4,ape_gi_29d435,1
5,ape_gi_42e115,1
6,ape_gi_856320,1
7,ape_gi_a10d1b,1
8,ape_inv_dcd836,1
9,ape_lh_d0adeb,1


In [6]:
useless_cols = list(useless_cols_df.column)
df = df.drop(columns=useless_cols + aggregated_cols).reset_index(drop=True)
display(df)

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,...,recency_hlthclaim_success,hlthclaim_cnt_unsuccess,recency_hlthclaim_unsuccess,flg_hlthclaim_839f8a_ever,recency_hlthclaim_839f8a,flg_hlthclaim_14cb37_ever,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,f_purchase_lh
0,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,1974-05-09,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,1979-11-11,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
2,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,1976-01-28,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
3,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,1976-03-19,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
4,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,1995-07-31,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17987,3363260248,None,Singapore,P,LAPSED,2016-08-01,1994-05-24,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
17988,c3509f0aef,Chinese,Singapore,P,ACTIVE,1990-03-20,1948-12-16,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
17989,6e44cfb3e7,Chinese,Singapore,P,ACTIVE,2007-12-31,1967-12-06,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
17990,dd4e362f9f,Others,Singapore,P,ACTIVE,2007-09-01,1986-04-01,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


# 2. Casting of Data Types

In [7]:
numerical_check = (metadf.is_numeric == 1) & (metadf.is_date == 0)
numerical_cols = metadf.loc[numerical_check].column_name.to_list()

date_check = (metadf.is_numeric == 1) & (metadf.is_date == 1)
date_cols = metadf.loc[date_check].column_name.to_list()

categorical_check = metadf.is_numeric == 0
categorical_cols = metadf.loc[categorical_check].column_name.to_list()

len(numerical_cols) + len(date_cols) + len(categorical_cols) == df.shape[1]

True

In [8]:
# further split categorical columns to 1. cat_cols: columns that do not require OHE, 2. cat_ohe_cols: columns that require OHE
categorical_regex = re.compile('^((f_)|(flg_)|(is_))')
cat_cols = [col for col in categorical_cols if categorical_regex.match(col)]
cat_ohe_cols = list(set(categorical_cols) - set(cat_cols)) # to create OHE under Section (4)
cat_ohe_cols.remove("clntnum") # remove PK

len(categorical_cols) == len(cat_cols) + len(cat_ohe_cols) + 1 # 1 for PK

True

In [9]:
df[numerical_cols] = df[numerical_cols].astype(float)
df[date_cols] = df[date_cols].apply(lambda x: pd.to_datetime(x, errors='coerce', format='%Y-%m-%d'))
df[cat_cols] = df[cat_cols].apply(pd.to_numeric, errors='coerce')

In [10]:
# remaining columns for OHE
cat_ohe_cols

['stat_flag',
 'clttype',
 'race_desc',
 'hh_size_est',
 'cltsex_fix',
 'annual_income_est',
 'ctrycode_desc']

# 3. Remove Rows
- Unreasonable values:  
    1. n_months_last_bought_products contains -ve values

In [11]:
unreasonable_negative_values = df.n_months_last_bought_products < 0

In [12]:
df = df.loc[~unreasonable_negative_values].\
        reset_index(drop=True)
display(df)

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,...,recency_hlthclaim_success,hlthclaim_cnt_unsuccess,recency_hlthclaim_unsuccess,flg_hlthclaim_839f8a_ever,recency_hlthclaim_839f8a,flg_hlthclaim_14cb37_ever,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,f_purchase_lh
0,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,1974-05-09,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,1979-11-11,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,1976-01-28,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,1976-03-19,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,1995-07-31,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17979,3363260248,None,Singapore,P,LAPSED,2016-08-01,1994-05-24,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17980,c3509f0aef,Chinese,Singapore,P,ACTIVE,1990-03-20,1948-12-16,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17981,6e44cfb3e7,Chinese,Singapore,P,ACTIVE,2007-12-31,1967-12-06,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17982,dd4e362f9f,Others,Singapore,P,ACTIVE,2007-09-01,1986-04-01,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 4. Feature Engineering
1. Replace all countries except for "Singapore" as "Others" in `ctrycode_desc`
2. Derive `num_months_since_first_interaction` from `min_occ_date`
3. Derive `client_age` from `cltdob_fix`
4. Encode categorical columns with OHE

In [13]:
date_today = dt.datetime.now()
print(date_today)

2024-01-28 01:14:27.950133


In [14]:
# ctrycode_desc: group to SG and others
df.loc[df.ctrycode_desc != "Singapore", 'ctrycode_desc'] = "Others"

# getting number of months since first interaction
year_diff = date_today.year - df["min_occ_date"].dt.year # To-do
month_diff = date_today.month - df["min_occ_date"].dt.month
df["num_months_since_first_interaction"] = year_diff * 12 + month_diff

# getting clients' age
df["client_age"] = date_today.year - df["cltdob_fix"].dt.year

In [15]:
# OHE
ohe_cols = ["clttype", "stat_flag", "race_desc"]
binary_cols = ["cltsex_fix", "ctrycode_desc"]
ordinal_cols = ["annual_income_est", "hh_size_est"]

In [16]:
unique_col_values = {}

print("OHE columns")
for col in ohe_cols:
    unique_values = sorted(df[col].unique().tolist())
    unique_values = [value for value in unique_values if value != None]
    unique_col_values[col] = unique_values
    print(col, unique_values)

print("\nBinary columns")
for col in binary_cols:
    unique_values = sorted(df[col].unique().tolist())
    unique_values = [value for value in unique_values if value != None]
    unique_col_values[col] = unique_values
    print(col, unique_values)

print("\nOrdinal categorical columns")
for col in ordinal_cols:
    unique_values = sorted(df[col].unique().tolist())
    unique_values = [value for value in unique_values if value != None]
    unique_col_values[col] = unique_values
    print(col, unique_values)

OHE columns
clttype ['P', 'G', 'C']
stat_flag ['ACTIVE', 'LAPSED', 'MATURED']
race_desc ['Chinese', 'Others', 'Indian', 'Malay']

Binary columns
cltsex_fix ['Female', 'Male']
ctrycode_desc ['Singapore', 'Others']

Ordinal categorical columns
annual_income_est ['C.60K-100K', 'D.30K-60K', 'A.ABOVE200K', 'B.100K-200K', 'E.BELOW30K']
hh_size_est ['1', '3', '4', '>4', '2', '0']


In [17]:
def map_annual_income_est(s):
    if s == None: 
        return None
    elif s.startswith('A'):
        return 1
    elif s.startswith('B'):
        return 2
    elif s.startswith('C'):
        return 3
    elif s.startswith('D'):
        return 4
    elif s.startswith('E'):
        return 5
    else:
        print(s)
        assert False # This LoC should never run
    
def map_hh_size_est(s):
    if s == None:
        return None
    elif s == '>4':
        return 5
    assert int(s) in list(range(5))
    return int(s)

In [18]:
for col in ohe_cols:
    unique_values = unique_col_values[col]
    for value in unique_values:
        new_col_name = f"{col}_{value}"
        df[new_col_name] = df.apply(lambda row: '' if row[col] is None else int(row[col] == value), axis=1)

In [19]:
for col in binary_cols:
    unique_values = unique_col_values[col]
    new_col_name = f"is_{unique_values[0]}"
    df[new_col_name] = df.apply(lambda row: '' if row[col] is None else int(row[col] == unique_values[0]), axis=1)

In [20]:
df["annual_income_est"] = df["annual_income_est"].map(map_annual_income_est)
df["hh_size_est"] = df["hh_size_est"].map(map_hh_size_est)

In [21]:
df = df.drop(columns=ohe_cols)
df = df.drop(columns=binary_cols)

In [22]:
df

,clntnum,min_occ_date,cltdob_fix,flg_substandard,flg_is_borderline_standard,flg_is_revised_term,flg_is_rental_flat,flg_has_health_claim,flg_has_life_claim,flg_gi_claim,...,clttype_C,stat_flag_ACTIVE,stat_flag_LAPSED,stat_flag_MATURED,race_desc_Chinese,race_desc_Others,race_desc_Indian,race_desc_Malay,is_Female,is_Singapore
0,91b546e924,2017-10-31,1974-05-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1,0,0,0,1,1
1,896bae548c,2007-05-23,1979-11-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1,0,0,0,0,1
2,f364439ae6,2019-08-31,1976-01-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,1,0,0,0,1
3,70f319cfe1,2021-10-18,1976-03-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1,0,0,0,1,1
4,2647a81328,2018-07-20,1995-07-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17979,3363260248,2016-08-01,1994-05-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,,,,,0,1
17980,c3509f0aef,1990-03-20,1948-12-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1,0,0,0,0,1
17981,6e44cfb3e7,2007-12-31,1967-12-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1,0,0,0,1,1
17982,dd4e362f9f,2007-09-01,1986-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,1,0,0,0,1


# 5. Handling of Missing Data

aengus plz help tolong tolong 